In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

Partie match

In [2]:
eve = input()
url = "https://www.vlr.gg/event/matches/" + eve + "/?series_id=all"
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

In [3]:
t_ns = soup.find_all('div', class_={'match-item-vs-team-name'})
team_name = [t_n.get_text(strip=True) for t_n in t_ns]

t_ss = soup.find_all('div', class_={'match-item-vs-team-score'})
team_score = [t_s.get_text(strip=True) for t_s in t_ss]

all_event = soup.find_all('div', class_={'match-item-event'})
event = [' '.join(all_e.stripped_strings) for all_e in all_event]

matchs = []
num_mat = []
for link in soup.find_all('a', class_={'wf-module-item'}):
    matchs.append(link.get('href'))
    
for item in matchs:
    match = re.match(r'/(?P<number>\d+)/(?P<text>.+)', item)
    if match:
        num_mat.append(match.group(1))

df = pd.DataFrame({'Num_Match':num_mat, 'Event':event, 'Team1': team_name[::2], 'Score Team1': team_score[::2], 'Team2': team_name[1::2], 'Score Team2': team_score[1::2]})
df

,Num_Match,Event,Team1,Score Team1,Team2,Score Team2
0,291311,Quarterfinals Open Qualifier,GoNext Esports,13,HASBYGANG,0
1,291312,Quarterfinals Open Qualifier,Szypko,11,Hamburger Jungs,13
2,291313,Quarterfinals Open Qualifier,Galakticos,13,Cicadas Gaming,5
3,291314,Quarterfinals Open Qualifier,Zerance,15,BRATAN MIX,17
4,291315,Semifinals Open Qualifier,GoNext Esports,2,Hamburger Jungs,0
5,291316,Semifinals Open Qualifier,Galakticos,2,BRATAN MIX,0
6,291317,Grand Final Open Qualifier,GoNext Esports,2,Galakticos,0
7,291448,Round 1 Group Stage,Fire Flux Esports,2,Solary,0
8,291449,Round 1 Group Stage,DSYRE,1,Mandatory,1
9,291450,Round 1 Group Stage,Acend,2,SAW,0


In [4]:
set(df['Team1'].unique()).union(df['Team2'].unique())

{'Acend',
 'BRATAN MIX',
 'CGN Esports',
 'Cicadas Gaming',
 'DSYRE',
 'DVM Esport',
 'Fire Flux Esports',
 'Galakticos',
 'GoNext Esports',
 'HASBYGANG',
 'Hamburger Jungs',
 'Mandatory',
 'NASR Esports',
 'SAW',
 'Solary',
 'Szypko',
 'UCAM Esports Club',
 'Zerance'}

Partie stats

In [5]:
url = "https://www.vlr.gg/event/stats/"+ eve
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

In [6]:
all_td = soup.find_all('td')
cell_content = [td.get_text(strip=True) for td in all_td]
# print(cell_content)
# Diviser les données en lignes avec 20 colonnes par ligne
num_columns = 21
table_data = [cell_content[i:i+num_columns] for i in range(0, len(cell_content), num_columns)]

# Créer un DataFrame pandas
df = pd.DataFrame(table_data, columns=[f'Col{i+1}' for i in range(num_columns)])
df = df.drop(['Col2'], axis=1)
# Afficher le DataFrame
df

,Col1,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12,Col13,Col14,Col15,Col16,Col17,Col18,Col19,Col20,Col21
0,TajReXSzyp,24,1.37,278.0,1.44,83%,173.9,0.96,0.63,0.08,0.00,21%,,0/2,23,23,16,15,2,0
1,LuvjixSzyp,24,1.34,204.0,1.38,83%,126.6,0.75,0.50,0.08,0.08,32%,,0/7,18,18,13,12,2,2
2,zeekGN,173,1.29,275.2,1.38,82%,175.9,0.94,0.32,0.14,0.05,22%,6%,1/16,25,163,118,55,25,9
3,jardakuzelGN,91,1.28,196.8,1.47,85%,137.0,0.76,0.26,0.09,0.04,32%,43%,3/7,16,69,47,24,8,4
4,grubinhoUCAM,150,1.25,229.4,1.36,73%,145.0,0.83,0.39,0.10,0.06,33%,8%,2/24,25,124,91,59,15,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,hypeRRHASB,13,0.38,135.0,0.46,54%,75.4,0.46,0.08,0.08,0.23,11%,,0/1,6,6,13,1,1,3
88,appAHASB,13,0.30,107.0,0.38,38%,80.0,0.38,0.00,0.08,0.08,38%,,0/1,5,5,13,0,1,1
89,yTGAL,15,0.22,90.0,0.29,33%,57.7,0.27,0.07,0.07,0.07,27%,,0/1,4,4,14,1,1,1
90,AtomHASB,13,0.21,51.0,0.15,38%,33.7,0.15,0.15,0.00,0.08,7%,,0/3,2,2,13,2,0,1


Partie stats agents

In [7]:
url = "https://www.vlr.gg/event/agents/"+ eve
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

In [8]:
soup

<!DOCTYPE html>

<html lang="en">
<head>
<title>
			Mandatory.gg Cup #3: Agent Pick Rates and Compositions | Valorant Event | VLR.gg		</title>
<meta content="Mandatory.gg Cup #3 agent pick rates by map and team" name="description"/>
<meta charset="utf-8"/>
<meta content="Mandatory.gg Cup #3: Agent Pick Rates and Compositions | Valorant Event | VLR.gg" property="og:title"/>
<meta content="VLR.gg" property="og:site_name"/>
<meta content="Mandatory.gg Cup #3 agent pick rates by map and team" property="og:description"/>
<meta content="/img/vlr/favicon.png" property="og:image"/>
<meta content="summary" name="twitter:card"/>
<meta content="@vlrdotgg" name="twitter:site"/>
<meta content="Mandatory.gg Cup #3: Agent Pick Rates and Compositions | Valorant Event | VLR.gg" name="twitter:title"/>
<meta content="Mandatory.gg Cup #3 agent pick rates by map and team" name="twitter:description"/>
<meta content="https://www.vlr.gg/img/vlr/logo_tw.png" name="twitter:image"/>
<meta content="#000" name="th

In [9]:
target_table = soup.find('table', class_='wf-table mod-pr-global')

# Vérifier si la table a été trouvée
if target_table:
    # Trouver toutes les balises <td> dans la table
    all_td = target_table.find_all('td')

    # Extraire le texte de chaque cellule
    cell_contents = [td.get_text(strip=True) for td in all_td]
# Diviser les données en lignes avec 20 colonnes par ligne
num_columns = 27
table_data = [cell_contents[i:i+num_columns] for i in range(0, len(cell_contents), num_columns)]

header_row = soup.find('div', class_='pr-matrix-map')
colums_headers = header_row.find_all('th') if header_row else []

title_list = ["Map", "Pick", "ATK", "DEF"]
for header in colums_headers:
    img_tag = header.find('img')
    if img_tag:
        title_attribute = img_tag.get('title')
        title_list.append(title_attribute)

# Créer un DataFrame pandas
df = pd.DataFrame(table_data, columns=title_list)
df['Map'] = df['Map'].str[1:]

df

,Map,Pick,ATK,DEF,Jett,Omen,Skye,Viper,Raze,Sova,...,Phoenix,Neon,Harbor,Sage,Fade,Yoru,Iso,Reyna,Deadlock,Chamber
0,,42,49%,51%,61%,61%,49%,48%,45%,44%,...,5%,4%,4%,4%,2%,1%,0%,0%,0%,0%
1,Haven,11,47%,53%,100%,95%,27%,27%,0%,73%,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
2,Bind,7,48%,52%,7%,0%,86%,93%,100%,7%,...,0%,0%,7%,0%,14%,7%,0%,0%,0%,0%
3,Ascent,6,50%,50%,100%,75%,0%,0%,0%,100%,...,25%,0%,0%,0%,0%,0%,0%,0%,0%,0%
4,Split,6,54%,46%,58%,58%,75%,33%,83%,0%,...,0%,8%,0%,25%,0%,0%,0%,0%,0%,0%
5,Lotus,5,57%,43%,20%,90%,100%,70%,90%,0%,...,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
6,Breeze,4,43%,57%,75%,0%,25%,100%,0%,75%,...,0%,25%,25%,0%,0%,0%,0%,0%,0%,0%
7,Sunset,3,42%,58%,17%,83%,33%,33%,83%,33%,...,17%,0%,0%,0%,0%,0%,0%,0%,0%,0%


In [10]:
dfs = {}
dfs_team = {}
maps = []
matrixs = soup.find_all('div', class_='pr-matrix-map') #6 matrices

for a in range(len(matrixs)):
    list = []
    for row in matrixs[a].find_all('tr', class_='pr-matrix-row'):
        if 'mod-dropdown' not in row.get('class', []):
            for cell in row.find_all('td'):
                value = 1 if 'mod-picked' in cell.get('class', []) else 0
                list.append(value)
        
    el = 25
    title_list= ["A","B"]
    for header in colums_headers:
        img_tag = header.find('img')
        if img_tag:
            title_attribute = img_tag.get('title')
            title_list.append(title_attribute)
    sous_listes = [list[i:i+el] for i in range(0, len(list), el)]
    dfs[a] = pd.DataFrame(sous_listes, columns=title_list)

    th_tags = matrixs[a].find_all('th')
    for th_tag in th_tags:
        text = th_tag.get_text(strip=True)
        maps.append(text[1:])
    while('' in maps):
        maps.remove('')
    df_maps = pd.DataFrame(maps)

teams = []
for x in range(len(matrixs[0].find_all('span', class_="text-of"))):
    teams.append(matrixs[0].find_all('span', class_="text-of")[x].get_text(strip=True))
    df_team = pd.DataFrame(teams)

for e in range(len(dfs)):
    exec(f'matrice_{e} =  dfs[{e}]')
    #exec(f'df{e} = df_team.insert(maps[e])')
    exec(f'df{e} = pd.concat([df_team,matrice_{e}], axis=1)')
    exec(f"df{e}.rename(columns={{df{e}.columns[0]: maps[{e}]}}, inplace=True)")
    exec(f'df{e} = df{e}.drop(["A","B"], axis=1)')

In [11]:
df0

,Split,Jett,Omen,Skye,Viper,Raze,Sova,Killjoy,Cypher,Kayo,...,Phoenix,Neon,Harbor,Sage,Fade,Yoru,Iso,Reyna,Deadlock,Chamber
0,Acend,1,1,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
1,CGN Esports,0,0,1,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
2,GoNext Esports,0,1,1,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,Galakticos,1,1,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,UCAM Esports Club,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,NASR Esports,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,DVM Esport,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Solary,0,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
8,Fire Flux Esports,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,SAW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
df1

,Haven,Jett,Omen,Skye,Viper,Raze,Sova,Killjoy,Cypher,Kayo,...,Phoenix,Neon,Harbor,Sage,Fade,Yoru,Iso,Reyna,Deadlock,Chamber
0,Acend,1,1,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CGN Esports,1,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,GoNext Esports,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Galakticos,1,1,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,UCAM Esports Club,1,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,NASR Esports,1,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,DVM Esport,1,1,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Solary,1,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Fire Flux Esports,1,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9,SAW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df2

,Breeze,Jett,Omen,Skye,Viper,Raze,Sova,Killjoy,Cypher,Kayo,...,Phoenix,Neon,Harbor,Sage,Fade,Yoru,Iso,Reyna,Deadlock,Chamber
0,Acend,1,0,0,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,CGN Esports,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,GoNext Esports,1,0,0,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,Galakticos,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,UCAM Esports Club,1,0,0,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
5,NASR Esports,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,DVM Esport,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Solary,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Fire Flux Esports,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,SAW,0,0,1,1,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0


In [14]:
df3

,Bind,Jett,Omen,Skye,Viper,Raze,Sova,Killjoy,Cypher,Kayo,...,Phoenix,Neon,Harbor,Sage,Fade,Yoru,Iso,Reyna,Deadlock,Chamber
0,Acend,0,0,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,CGN Esports,0,0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,GoNext Esports,0,0,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Galakticos,0,0,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,UCAM Esports Club,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
5,NASR Esports,0,0,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,DVM Esport,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Solary,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Fire Flux Esports,0,0,0,1,1,0,0,0,1,...,0,0,1,0,1,0,0,0,0,0
9,SAW,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df4

,Ascent,Jett,Omen,Skye,Viper,Raze,Sova,Killjoy,Cypher,Kayo,...,Phoenix,Neon,Harbor,Sage,Fade,Yoru,Iso,Reyna,Deadlock,Chamber
0,Acend,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CGN Esports,1,0,0,0,0,1,1,0,0,...,1,0,0,0,0,0,0,0,0,0
2,GoNext Esports,1,1,0,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Galakticos,1,1,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,UCAM Esports Club,1,1,0,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5,NASR Esports,1,1,0,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
6,DVM Esport,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Solary,1,1,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
8,Fire Flux Esports,1,1,0,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
9,SAW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
df5

,Lotus,Jett,Omen,Skye,Viper,Raze,Sova,Killjoy,Cypher,Kayo,...,Phoenix,Neon,Harbor,Sage,Fade,Yoru,Iso,Reyna,Deadlock,Chamber
0,Acend,0,0,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CGN Esports,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,GoNext Esports,0,1,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Galakticos,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,UCAM Esports Club,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,NASR Esports,0,1,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,DVM Esport,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Solary,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Fire Flux Esports,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,SAW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df6

,Sunset,Jett,Omen,Skye,Viper,Raze,Sova,Killjoy,Cypher,Kayo,...,Phoenix,Neon,Harbor,Sage,Fade,Yoru,Iso,Reyna,Deadlock,Chamber
0,Acend,0,1,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,CGN Esports,0,1,1,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
2,GoNext Esports,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Galakticos,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,UCAM Esports Club,0,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,NASR Esports,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,DVM Esport,1,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,Solary,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Fire Flux Esports,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,SAW,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
